<a href="https://colab.research.google.com/github/Lakshmiaishwarya01/Sproutify/blob/main/sproutifyml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numpy scikit-learn firebase-admin


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 458.0/458.0 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.2/77.2 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 54.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of cachecontrol to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of google-api-core to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of google-api-core to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. S

In [16]:
import pandas as pd

# Define the dataset
data = {
    "temperature": [24, 26, 25, 27, 28, 23, 22],
    "humidity": [65, 70, 75, 60, 55, 80, 85],
    "soil_moisture": [30, 28, 32, 25, 20, 35, 40],
    "water_requirement": [150, 170, 160, 180, 200, 140, 130],
    "nutrient_requirement": [40, 42, 38, 45, 50, 35, 30],
     "plant": ["money_plant", "money_plant", "money_plant", "money_plant", "money_plant", "money_plant", "money_plant"]

}

# Convert to DataFrame
df = pd.DataFrame(data)

# Save as a CSV file
df.to_csv("money_plant_dataset.csv", index=False)

print("Dataset saved as money_plant_dataset.csv")


Dataset saved as money_plant_dataset.csv


In [18]:
import firebase_admin
from firebase_admin import credentials, db
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import joblib

# Step 1: Firebase Setup with firebase-admin
if not firebase_admin._apps:
  cred = credentials.Certificate("/firebase-sdk.json")
  firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://sproutify-3a23c-default-rtdb.asia-southeast1.firebasedatabase.app/'
})
else:
    # Use the existing default app
    print("Default Firebase app already initialized. Using existing app.")


# Step 2: Load CSV Dataset for Training
data = pd.read_csv("money_plant_dataset.csv")

# Encode plant type (one-hot encoding)
data = pd.get_dummies(data, columns=["plant"])

# Features and targets
X = data[["temperature", "humidity", "soil_moisture", "plant_money_plant"]]
y_water = data["water_requirement"]
y_nutrient = data["nutrient_requirement"]

# Train-test split
X_train, X_test, y_water_train, y_water_test = train_test_split(X, y_water, test_size=0.2, random_state=42)
_, _, y_nutrient_train, y_nutrient_test = train_test_split(X, y_nutrient, test_size=0.2, random_state=42)

# Train models
water_model = LinearRegression()
nutrient_model = LinearRegression()

water_model.fit(X_train, y_water_train)
nutrient_model.fit(X_train, y_nutrient_train)

# Save models
joblib.dump(water_model, "water_model.pkl")
joblib.dump(nutrient_model, "nutrient_model.pkl")

# Step 3: Firebase Integration for Real-Time Predictions
def fetch_sensor_data():
    """
    Fetches real-time sensor data from Firebase.
    Assumes a structure where sensor data is stored under "sensors".
    """
    sensor_data_ref = db.reference("sensors")
    sensor_data = sensor_data_ref.get()
    return sensor_data

def store_predictions(plant_type, water_req, nutrient_req):
    """
    Stores prediction results in Firebase under the "predictions" node.
    """
    predictions_ref = db.reference("predictions")
    predictions_ref.set({
        "plant": plant_type,
        "water_requirement": water_req,
        "nutrient_requirement": nutrient_req
    })

# Step 4: Real-Time Prediction Logic
def predict_requirements(sensor_data, plant_type):
    """
    Uses trained models to predict water and nutrient requirements based on sensor data.
    """
    # Load trained models
    water_model = joblib.load("water_model.pkl")
    nutrient_model = joblib.load("nutrient_model.pkl")

    # Prepare input data for prediction
    input_data = np.array([[
        sensor_data["temperature"],
        sensor_data["humidity"],
        sensor_data["soil_moisture"],
        1 if plant_type == "money_plant" else 0,

    ]])

    # Make predictions
    water_req = water_model.predict(input_data)[0]
    nutrient_req = nutrient_model.predict(input_data)[0]

    return water_req, nutrient_req

# Fetch data from Firebase, predict, and store results
sensor_data = fetch_sensor_data()  # Fetch real-time sensor data
plant_type = sensor_data["plant"]  # e.g., "money_plant" or "basil"

# Make predictions
water_req, nutrient_req = predict_requirements(sensor_data, plant_type)

# Store predictions in Firebase
store_predictions(plant_type, water_req, nutrient_req)

print(f"Water Requirement: {water_req:.2f} mL, Nutrient Requirement: {nutrient_req:.2f} mg")


Default Firebase app already initialized. Using existing app.


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


Water Requirement: 161.00 mL, Nutrient Requirement: 40.00 mg
